In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
import cv2
from tqdm import tqdm
from collections import deque
import wave, struct
import cProfile, re
#cProfile.run('re.compile("foo|bar")')

In [2]:
WIDTH = int(720)

def normilizeFrame(frame):
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    #cv2.imwrite("gray.png", frame)
    frame = cv2.resize(frame, (WIDTH, frame.shape[0]), interpolation=cv2.INTER_NEAREST)
    return frame / 255.0

fileName = 'testfiles/Mind.In.A.Box - I Love 64'

cap = cv2.VideoCapture(fileName + '.avi')
if (cap.isOpened()== False): 
  print("Error opening video stream or file")

ret, frame = cap.read()
if not ret:
    print('error reading frame')

gray = normilizeFrame(frame)

In [3]:
height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
frameCount = cap.get(cv2.CAP_PROP_FRAME_COUNT)

print(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
print(height)
print(frameCount)

141.0
625.0
4908.0


In [4]:
model = keras.models.load_model("model")

2022-02-24 10:54:08.098584: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-02-24 10:54:08.858255: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 586 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1050, pci bus id: 0000:02:00.0, compute capability: 6.1


In [5]:
def deinterlase(frame):
    even_fields=frame[::2,:];
    odd_fields=frame[1::2,:];
    return np.concatenate((even_fields, odd_fields), axis=0)

bufL0 = deque()
bufR0 = deque()
bufL1 = deque()
bufR1 = deque()
bufL2 = deque()
emptyLine = np.zeros(14, dtype=bool)

#kluge init queues
for i in range((16*5)-1):
    bufL0.append(emptyLine)

for i in range((16*4)-1):
    bufR0.append(emptyLine)

for i in range((16*3)-1):
    bufL1.append(emptyLine)

for i in range((16*2)-1):
    bufR1.append(emptyLine)
    
for i in range(16-1):
    bufL2.append(emptyLine)

bitsToInt = lambda ls: int("".join([str(int(b)) for b in ls]), 2)
    
def deinterlive(block):
    block = block[4:88]
    
    bufL0.append(block[:14])
    bufR0.append(block[14:28])
    bufL1.append(block[28:42])
    bufR1.append(block[42:56])
    bufL2.append(block[56:70])
    
    return bufL0.popleft(), bufR0.popleft(), bufL1.popleft(), bufR1.popleft(), bufL2.popleft(), block[70:84] #L0, R0, L1, R1, L2, R2

In [6]:
bar = tqdm(total=frameCount - (cap.get(cv2.CAP_PROP_POS_FRAMES) -1))
bar.set_description("decoding")
file = wave.open(fileName + '.wav', mode='w')
file.setnchannels(2) 
file.setsampwidth(2)
file.setframerate(44100)

decoding:   0%|                                                                                  | 0/4908.0 [00:00<?, ?it/s]

In [7]:
def a():
    while True:
        bar.update(1)
        if cap.grab():
            ret, frame = cap.retrieve()
            
            if not ret:
                print('error reading frame')
                break
                    
            gray = normilizeFrame(frame)
            #cv2.imwrite("gray_norm.png", gray)
            
            gray = deinterlase(gray)
            
            #cv2.imwrite("gray_deint.png", gray)
            service, data = model.predict(gray, verbose=0)
            service = service > 0.5
            data = data > 0.5
            
            audioFrame = bytearray()
            
            for i in range(len(service)):
                if service[i][0]:
                    L0, R0, L1, R1, L2, R2 = deinterlive(data[i])
                    audioFrame += struct.pack('<H', (bitsToInt(L0) << 2))
                    audioFrame += struct.pack('<H', (bitsToInt(R0) << 2))
                    audioFrame += struct.pack('<H', (bitsToInt(L1) << 2))
                    audioFrame += struct.pack('<H', (bitsToInt(R1) << 2))
                    audioFrame += struct.pack('<H', (bitsToInt(L2) << 2))
                    audioFrame += struct.pack('<H', (bitsToInt(R2) << 2))
            
            keras.backend.clear_session()
            file.writeframes(audioFrame)
        if cap.get(cv2.CAP_PROP_POS_FRAMES) == frameCount:
            break

In [8]:
cProfile.run('a()')

decoding:   5%|███▍                                                                    | 233/4908.0 [00:48<17:04,  4.56it/s]

KeyboardInterrupt: 

decoding:   5%|███▍                                                                    | 233/4908.0 [02:12<44:19,  1.76it/s]